In [3]:
!pip3 install opacus

     |████████████████████████████████| 114 kB 735 kB/s eta 0:00:01
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import accuracy_score 

In [5]:
batch_size = 50

In [6]:
class ChurnDataset(Dataset):
 
    def __init__(self, csv_file):
  
        df = pd.read_csv(csv_file)
        
        df = df.drop(["Surname", "CustomerId", "RowNumber"], axis=1)

        # Grouping variable names
        self.categorical = ["Geography", "Gender"]
        self.target = "Exited"

        # One-hot encoding of categorical variables
        self.churn_frame = pd.get_dummies(df, prefix=self.categorical)

        # Save target and predictors
        self.X = self.churn_frame.drop(self.target, axis=1)
        self.y = self.churn_frame["Exited"]
        
        
        scaler = StandardScaler()
        X_array  = scaler.fit_transform(self.X)
        self.X = pd.DataFrame(X_array)

    def __len__(self):
        return len(self.churn_frame)

    def __getitem__(self, idx):
        # Convert idx from tensor to list due to pandas bug (that arises when using pytorch's random_split)
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()

        return [self.X.iloc[idx].values, self.y[idx]]

In [7]:
def get_CHURN_model():
    model = nn.Sequential(nn.Linear(13, 64), 
                    nn.ReLU(), 
                    nn.Linear(64, 64), 
                    nn.ReLU(), 
                    nn.Linear(64, 1)) 
    return model

In [8]:
def get_dataloader(csv_file, batch_size):
     # Load dataset
    dataset = ChurnDataset(csv_file)

    # Split into training and test
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    trainset, testset = random_split(dataset, [train_size, test_size])
    
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
    
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)
    
    return trainloader, testloader, trainset, testset

In [9]:
def train(trainloader, net, optimizer, n_epochs=100):
     
    device = "cpu"

    # Define the model
    #net = get_CHURN_model()
    net = net.to(device)
    
    #criterion = nn.CrossEntropyLoss() 
    criterion = nn.BCEWithLogitsLoss()


    # Train the net
    loss_per_iter = []
    loss_per_batch = []
    for epoch in range(n_epochs):

        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = net(inputs.float())
            loss = criterion(outputs, labels.float().unsqueeze(1))
            loss.backward()
            optimizer.step()

            # Save loss to plot
            running_loss += loss.item()
            loss_per_iter.append(loss.item())

        
        print("Epoch {} - Training loss: {}".format(epoch, running_loss/len(trainloader))) 
        
        running_loss = 0.0
        
    return net


In [10]:
csv_file = "data/churn.csv"

trainloader, testloader, train_ds, test_ds = get_dataloader(csv_file, batch_size)

In [11]:
net = get_CHURN_model()

optimizer = optim.Adam(net.parameters(), weight_decay=0.0001, lr=0.003)

model = train(trainloader, net, optimizer, 50)

Epoch 0 - Training loss: 0.42998170768842103
Epoch 1 - Training loss: 0.3585398425348103
Epoch 2 - Training loss: 0.34484875975176693
Epoch 3 - Training loss: 0.33772834809497
Epoch 4 - Training loss: 0.3370456677861512
Epoch 5 - Training loss: 0.3331598186865449
Epoch 6 - Training loss: 0.32938209315761924
Epoch 7 - Training loss: 0.3280104883015156
Epoch 8 - Training loss: 0.3263300991617143
Epoch 9 - Training loss: 0.3237824782729149
Epoch 10 - Training loss: 0.3222683455329388
Epoch 11 - Training loss: 0.32311330642551184
Epoch 12 - Training loss: 0.3190927365794778
Epoch 13 - Training loss: 0.31648382991552354
Epoch 14 - Training loss: 0.3140755801461637
Epoch 15 - Training loss: 0.3146016987040639
Epoch 16 - Training loss: 0.3130694762803614
Epoch 17 - Training loss: 0.31548011796548964
Epoch 18 - Training loss: 0.3086287043057382
Epoch 19 - Training loss: 0.3070000804029405
Epoch 20 - Training loss: 0.30901680393144487
Epoch 21 - Training loss: 0.3056166450958699
Epoch 22 - Trai